In [34]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
import shapely
from pyproj import Proj, Transformer

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['from_mother_road_direction'] = 'None'

buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [60]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def testLeftRightofLine(start,end,poi):
    d=(poi.x-start.x)*(end.y-start.y)-(poi.y-start.y)*(end.x-start.x)
    if d>0:
        #left
        return -1
    if d<0:
        #right
        return 1
    else:
        return 0

def findMotherRoad2AssociatedRoadDirection(mother_road,associated_road):
    left_buffered_line=mother_road.buffer(0.0001,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(associated_road.buffer(0.00001)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
    return direction

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        mother_road_direction=findMotherRoad2AssociatedRoadDirection(mother_road.geometry,associated_road.geometry)
        if (mother_road_direction == 1):
            # print("Right Direction")
            buildings.at[index,'from_mother_road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(geom_transformed.length)))+"/"
        if (mother_road_direction == -1):
            # print("left Direction")
            buildings.at[index,'from_mother_road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(geom_transformed.length)))+"/"
            
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
        if ((line_row.start== None) and (line_row.end != None)) :
            road.at[index_road,'mother_road']='end'
    
        if ((line_row.start!= None) and (line_row.end != None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
            mother_road_code=row.end
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        current_road = row.geometry
        mother_road_geom=mother_road
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
#         print(road_geom)
        start_point=mother_road_geom.intersection(road_geom)
        print(start_point)
        st=str(type(start_point))
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
            print(type(start_point))
            print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            print(start_point)
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry,degree_length_for_building
    except:
        pass



    
                   


In [5]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    addStartEndPoint(start_point,end_point,index)
    CalculateBearing(start_point,end_point,index)
    FindIntersectedFeatures(start_point,end_point,row,index)
    scan_I_feature(row,index)
    AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [13]:
row_fetched=0
while row_fetched<len(road.index):
    start=row_fetched
    end = row_fetched+100
    
    for i in range(start,end):
        row=road.iloc[i]
        try:
            CalculateTouchLine(i,row)
            print((f"""i: {index}""")
        except:
            continue
    print(f"""Batch Done, {end}""")
print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")   

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
0
end
POINT (83.47182876258739 27.59323857200252)
POINT (83.47182876258739 27.59323857200252)
1
POINT (83.47268607010614 27.60246986818993)
POINT (83.47268607010614 27.60246986818993)
2
POINT (83.47040628242405 27.57787412609008)
POINT (83.47040628242405 27.57787412609008)
3
POINT (83.47172783257349 27.59225408505802)
POINT (83.47172783257349 27.59225408505802)
4
end
POINT (83.46736454413843 27.56698659255727)
POINT (83.46736454413843 27.56698659255727)
5
POINT (83.47153400529015 27.59036561757927)
POINT (83.47153400529015 27.59036561757927)
6
end
POINT (83.47187124206425 27.59364565192533)
POINT (83.47187124206425 27.59364565192533)
7
end
POINT (83.47263962641767 27.60196558594311)
POINT (83.47263962641767 27.60196558594311)
8
POINT (83.47213499163661 27.59644264511201)
POINT (83.47213499163661 27.59644264511201)
9
POINT (83.47071132616685 27.58135272534804)
POINT (83.47071132616685 27.58135272534804)
1

start
POINT (83.45892516306589 27.56683897154136)
POINT (83.45892516306589 27.56683897154136)
89
POINT (83.47150689522709 27.59260663009195)
POINT (83.47150689522709 27.59260663009195)
90
POINT (83.4702157331696 27.59357258021043)
POINT (83.4702157331696 27.59357258021043)
91
POINT (83.46215994622457 27.57253243552356)
POINT (83.46215994622457 27.57253243552356)
92
start
POINT (83.46231292090465 27.56762419390145)
POINT (83.46231292090465 27.56762419390145)
93
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
94
POINT (83.46899543768933 27.59266431350733)
POINT (83.46899543768933 27.59266431350733)
95
POINT (83.46991239903804 27.58034128302387)
POINT (83.46991239903804 27.58034128302387)
96
end
POINT (83.46804093504119 27.57045325151637)
POINT (83.46804093504119 27.57045325151637)
97
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
98
start
POINT (83.46468126302995 27.57313549121)
POINT (83.46468126302995 27.57313549121

end
POINT (83.45892516306589 27.56683897154136)
POINT (83.45892516306589 27.56683897154136)
179
start
POINT (83.43251684727471 27.60744073140467)
POINT (83.43251684727471 27.60744073140467)
180
POINT (83.43736914746825 27.5958884028276)
POINT (83.43736914746825 27.5958884028276)
181
end
POINT (83.4548427625835 27.59128840923273)
POINT (83.4548427625835 27.59128840923273)
182
POINT (83.43558870385567 27.59570894581123)
POINT (83.43558870385567 27.59570894581123)
183
end
POINT (83.43478050381725 27.59557949020137)
POINT (83.43478050381725 27.59557949020137)
184
start
POINT (83.45268863357364 27.59713899556823)
POINT (83.45268863357364 27.59713899556823)
185
end
POINT (83.45336136153765 27.58646731821221)
POINT (83.45336136153765 27.58646731821221)
186
POINT (83.44703032768751 27.59477817627186)
POINT (83.44703032768751 27.59477817627186)
187
POINT (83.43686148377054 27.59586403299875)
POINT (83.43686148377054 27.59586403299875)
188
POINT (83.45430597434131 27.58973506691984)
POINT (83.45

POINT (83.46570238295868 27.56816980449292)
POINT (83.46570238295868 27.56816980449292)
268
end
POINT (83.46301775466571 27.59483154743799)
POINT (83.46301775466571 27.59483154743799)
269
end
MULTIPOINT (83.46700765987566 27.58456492653016, 83.46824549393443 27.58456295341762)
MULTIPOINT (83.46700765987566 27.58456492653016, 83.46824549393443 27.58456295341762)
270
POINT (83.46524160811379 27.56899020473423)
POINT (83.46524160811379 27.56899020473423)
271
POINT (83.46859447945172 27.57603419052555)
POINT (83.46859447945172 27.57603419052555)
272
POINT (83.46898322399664 27.57755337838574)
POINT (83.46898322399664 27.57755337838574)
273
POINT (83.46855191094198 27.58070076273009)
POINT (83.46855191094198 27.58070076273009)
274
POINT (83.46075547318662 27.56988600503513)
POINT (83.46075547318662 27.56988600503513)
275
POINT (83.4677398267317 27.57107695743571)
POINT (83.4677398267317 27.57107695743571)
276
POINT (83.46969148867134 27.58437466955542)
POINT (83.46969148867134 27.5843746695

POINT (83.45765913526094 27.59677169985412)
POINT (83.45765913526094 27.59677169985412)
355
POINT (83.43969072124986 27.5970122981791)
POINT (83.43969072124986 27.5970122981791)
356
POINT (83.45024668373753 27.59916563668486)
POINT (83.45024668373753 27.59916563668486)
357
POINT (83.45827922495104 27.60336004024421)
POINT (83.45827922495104 27.60336004024421)
358
POINT (83.45907372027602 27.5720712128163)
POINT (83.45907372027602 27.5720712128163)
359
POINT (83.45985714129409 27.57115195909569)
POINT (83.45985714129409 27.57115195909569)
360
POINT (83.44469313987656 27.59143003986452)
POINT (83.44469313987656 27.59143003986452)
361
POINT (83.44044875170857 27.5985606006912)
POINT (83.44044875170857 27.5985606006912)
362
POINT (83.44270290781299 27.59570364790505)
POINT (83.44270290781299 27.59570364790505)
363
POINT (83.46206107296018 27.60358919948203)
POINT (83.46206107296018 27.60358919948203)
364
POINT (83.45823347199718 27.60208084035855)
POINT (83.45823347199718 27.60208084035855

POINT (83.43361060994175 27.60681093347785)
POINT (83.43361060994175 27.60681093347785)
442
POINT (83.45861981445296 27.57372140761692)
POINT (83.45861981445296 27.57372140761692)
443
start
MULTIPOINT (83.43739226903807 27.59472507310369, 83.43741506865058 27.59541615802931)
MULTIPOINT (83.43739226903807 27.59472507310369, 83.43741506865058 27.59541615802931)
444
POINT (83.43421475021063 27.58619841012859)
POINT (83.43421475021063 27.58619841012859)
445
end
POINT (83.45048823714251 27.59394028691474)
POINT (83.45048823714251 27.59394028691474)
446
POINT (83.45217278874327 27.58071501158855)
POINT (83.45217278874327 27.58071501158855)
447
POINT (83.43457287913515 27.58667979563609)
POINT (83.43457287913515 27.58667979563609)
448
start
POINT (83.45084932214802 27.57466362290197)
POINT (83.45084932214802 27.57466362290197)
449
POINT (83.43468112873114 27.58679486838849)
POINT (83.43468112873114 27.58679486838849)
450
start
POINT (83.43860032384595 27.58323324012201)
POINT (83.438600323845

526
POINT (83.46523466624689 27.5877407798132)
POINT (83.46523466624689 27.5877407798132)
527
POINT (83.46450854643234 27.59885341994942)
POINT (83.46450854643234 27.59885341994942)
528
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
529
start
POINT (83.46386032229611 27.60548847861043)
POINT (83.46386032229611 27.60548847861043)
530
POINT (83.46454041300973 27.57080012092888)
POINT (83.46454041300973 27.57080012092888)
531
end
MULTIPOINT (83.46463900223237 27.59849377283165, 83.46597989395701 27.59799794794696)
MULTIPOINT (83.46463900223237 27.59849377283165, 83.46597989395701 27.59799794794696)
532
POINT (83.46464157684733 27.56881470383559)
POINT (83.46464157684733 27.56881470383559)
533
POINT (83.46512407121895 27.56931104057213)
POINT (83.46512407121895 27.56931104057213)
534
POINT (83.46795848789395 27.60418346519481)
POINT (83.46795848789395 27.60418346519481)
535
start
POINT (83.46795917947259 27.59296216447234)
POINT (83.46795917947259 27.592

end
POINT (83.42751669044299 27.60211535364073)
POINT (83.42751669044299 27.60211535364073)
613
end
POINT (83.45540072626255 27.60452921405204)
POINT (83.45540072626255 27.60452921405204)
614
end
POINT (83.46179254978642 27.59531565969002)
POINT (83.46179254978642 27.59531565969002)
615
LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
616
POINT (83.43811633749903 27.60427303317397)
POINT (83.43811633749903 27.60427303317397)
617
POINT (83.42104810528588 27.57221526082566)
POINT (83.42104810528588 27.57221526082566)
618
start
POINT (83.43917845392258 27.58777460601328)
POINT (83.43917845392258 27.58777460601328)
619
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
620
POINT (83.44001095184399 27.56725972385721)
POINT (83.44001095184399 27.56725972385721)
621
POINT (83.42957199115455 27.57817268212364)
POINT (83.42957199115455 27.57817268212364)
622
POINT (83.44705698269365 27.57860723453632)
POINT (83.44705698269365 27.57860723453632)
623
POINT (83.42

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
703
POINT (83.45112509744536 27.60052363545748)
POINT (83.45112509744536 27.60052363545748)
704
POINT (83.42440234898925 27.57122346329226)
POINT (83.42440234898925 27.57122346329226)
705
POINT (83.46695512867632 27.59193769007237)
POINT (83.46695512867632 27.59193769007237)
706
POINT (83.46780422718342 27.59278009222754)
POINT (83.46780422718342 27.59278009222754)
707
start
POINT (83.46533428325063 27.59733493625833)
POINT (83.46533428325063 27.59733493625833)
708
POINT (83.46771973857511 27.60410561718055)
POINT (83.46771973857511 27.60410561718055)
709
POINT (83.46477180317623 27.59008596349645)
POINT (83.46477180317623 27.59008596349645)
710
POINT (83.46401182658457 27.60624618980984)
POINT (83.46401182658457 27.60624618980984)
711
MULTIPOINT (83.465354005679 27.59703139413299, 83.46535863149569 27.59703751247127)
MULTIPOINT (83.465354005679 27.59703139413299, 83.46535863149569 27.59703751247127)
712

start
POINT (83.46701131202246 27.59352561851256)
POINT (83.46701131202246 27.59352561851256)
789
POINT (83.45126400230858 27.60045781613077)
POINT (83.45126400230858 27.60045781613077)
790
POINT (83.46784407699184 27.59454551180163)
POINT (83.46784407699184 27.59454551180163)
791
POINT (83.46423232236384 27.59021041527848)
POINT (83.46423232236384 27.59021041527848)
792
POINT (83.46574168513069 27.59074208997555)
POINT (83.46574168513069 27.59074208997555)
793
POINT (83.46572438397322 27.59616554690251)
POINT (83.46572438397322 27.59616554690251)
794
POINT (83.46442660110682 27.59015882656865)
POINT (83.46442660110682 27.59015882656865)
795
POINT (83.46553808401541 27.59653534632923)
POINT (83.46553808401541 27.59653534632923)
796
POINT (83.46780422718342 27.59278009222754)
POINT (83.46780422718342 27.59278009222754)
797
start
POINT (83.45341494404448 27.60283259984465)
POINT (83.45341494404448 27.60283259984465)
798
end
POINT (83.45398852355157 27.60496457675384)
POINT (83.4539885235

In [91]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point,degree_length_for_building=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue
#     print(roi_line)
#     print(shapely.geometry.mapping(roi_line))
    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       

    poi = row.geometry
    #constructing new geometry with new start and end for checking left and right 
    direction_updated_roi=LineString([start_roi,end_roi])

    left_buffered_line=direction_updated_roi.buffer(0.01,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(poi.buffer(0.00004)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
        
#     print(shapely.geometry.mapping(left_buffered_line))
#     print(shapely.geometry.mapping(poi.buffer(0.00004)))
    
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [93]:
i=2576
row=buildings.iloc[i]
assign_building_code(i,row)
print(buildings.iloc[i])

OBJECTID_1                                                                2577
NEW                                                                      TC075
geometry                       POINT Z (83.46912030226019 27.57323534743341 0)
road_direction                                                            left
distance2_intersecting_road                                               1238
road_length_d_code                                                        1239
Name: 2576, dtype: object


C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_21424/2332293344.py:311: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


In [15]:
row_fetched=0
while row_fetched<len(buildings.index):
    start=row_fetched
    end = row_fetched+100
    
    for i in range(start,end):
        row=buildings.iloc[i]
        try:
            assign_building_code(i,row)
            print(buildings.iloc[i].distance2_intersecting_road)
            print((f"""i: {index}""")
        except:
            continue
    print(f"""Batch Done, {end}""")
    row_fetched=end

C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_21424/1151795401.py:300: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


669
859
659
859
650
859
206
859
214
859
214
859
193
859
185
859
181
859
214
859
214
859
37
859
31
859
25
859
169
859
170
859
173
859
166
859
178
859
188
859
197
859
202
859
215
859
213
859
226
859
232
859
243
859
243
859
252
859
262
859
271
859
277
859
260
859
290
859
288
859
304
859
313
859
322
859
948
859
14
859
368
859
383
859
359
859
922
859
916
859
910
859
404
859
279
859
424
859
175
859
179
859
424
859
449
859
453
859
460
859
21
859
17
859
31
859
49
859
65
859
92
859
107
859
157
859
105
859
38
859
210
859
191
859
185
859
177
859
171
859
218
859
210
859
199
859
187
859
178
859
562
859
63
859
63
859
60
859
613
859
618
859
635
859
630
859
618
859
8
859
141
859
128
859
116
859
108
859
96
859
82
859
117
859
23
859
135
859
59
859
63
859
52
859
40
859
32
859
55
859
Batch Done, 100
71
859
100
859
21
859
15
859
1
859
1140
859
1151
859
1164
859
26
859
181
859
25
859
17
859
1062
859
1152
859
1179
859
1066
859
13
859
11
859
15
859
6
859
4
859
203
859
33
859
13
859
27
859
32
859
190
859
193
8

99
859
98
859
105
859
114
859
677
859
695
859
738
859
748
859
774
859
786
859
620
859
605
859
527
859
8
859
492
859
21
859
48
859
77
859
457
859
445
859
435
859
422
859
410
859
415
859
427
859
425
859
105
859
116
859
238
859
226
859
196
859
169
859
177
859
155
859
137
859
130
859
110
859
96
859
48
859
16
859
145
859
110
859
33
859
30
859
209
859
221
859
169
859
164
859
149
859
136
859
187
859
266
859
289
859
300
859
316
859
283
859
292
859
293
859
295
859
Batch Done, 1100
301
859
304
859
296
859
287
859
188
859
313
859
323
859
337
859
330
859
333
859
346
859
355
859
362
859
176
859
199
859
427
859
228
859
19
859
18
859
14
859
413
859
162
859
166
859
11
859
23
859
28
859
545
859
579
859
581
859
12
859
590
859
597
859
636
859
624
859
620
859
653
859
675
859
676
859
704
859
732
859
35
859
41
859
752
859
762
859
793
859
663
859
82
859
97
859
118
859
129
859
141
859
164
859
166
859
155
859
148
859
148
859
154
859
148
859
148
859
152
859
148
859
370
859
311
859
320
859
11
859
16
859
32
859
4

148
859
627
859
250
859
115
859
74
859
62
859
Batch Done, 2100
52
859
184
859
41
859
28
859
206
859
75
859
89
859
147
859
157
859
171
859
113
859
211
859
184
859
172
859
149
859
29
859
28
859
19
859
665
859
679
859
10
859
23
859
399
859
260
859
34
859
38
859
23
859
29
859
38
859
45
859
20
859
127
859
44
859
82
859
51
859
43
859
887
859
749
859
769
859
931
859
963
859
856
859
93
859
85
859
65
859
60
859
127
859
726
859
31
859
31
859
70
859
271
859
271
859
24
859
24
859
105
859
105
859
199
859
177
859
178
859
222
859
221
859
430
859
68
859
107
859
543
859
567
859
581
859
152
859
237
859
253
859
251
859
497
859
521
859
534
859
529
859
494
859
22
859
456
859
468
859
33
859
69
859
80
859
97
859
112
859
149
859
14
859
599
859
582
859
571
859
12
859
49
859
70
859
105
859
118
859
118
859
132
859
151
859
80
859
42
859
Batch Done, 2200
50
859
18
859
4
859
80
859
220
859
230
859
255
859
381
859
329
859
186
859
177
859
179
859
201
859
214
859
217
859
193
859
178
859
188
859
223
859
270
859
322
859

295
859
294
859
330
859
324
859
330
859
348
859
364
859
343
859
351
859
360
859
357
859
375
859
385
859
403
859
409
859
419
859
430
859
429
859
438
859
442
859
455
859
473
859
482
859
452
859
427
859
522
859
521
859
701
859
724
859
109
859
181
859
90
859
76
859
519
859
474
859
452
859
463
859
407
859
399
859
381
859
Batch Done, 3200
388
859
369
859
363
859
359
859
415
859
519
859
330
859
313
859
324
859
297
859
286
859
276
859
255
859
243
859
243
859
229
859
216
859
206
859
199
859
183
859
165
859
188
859
178
859
146
859
135
859
141
859
145
859
118
859
96
859
119
859
59
859
25
859
11
859
1035
859
1035
859
1063
859
1057
859
1055
859
130
859
192
859
224
859
247
859
219
859
179
859
427
859
401
859
964
859
969
859
972
859
795
859
784
859
765
859
763
859
747
859
751
859
722
859
720
859
703
859
692
859
88
859
231
859
310
859
339
859
356
859
449
859
432
859
341
859
341
859
1243
859
1269
859
1274
859
1267
859
1255
859
1238
859
535
859
526
859
124
859
161
859
216
859
231
859
289
859
302
859
311

217
859
216
859
215
859
222
859
240
859
242
859
258
859
267
859
280
859
293
859
302
859
313
859
1641
859
1650
859
1640
859
1621
859
1612
859
1605
859
1668
859
1664
859
1655
859
1644
859
1642
859
1636
859
1637
859
1628
859
1626
859
1595
859
1600
859
85
859
1581
859
10
859
1572
859
1571
859
1563
859
1558
859
1557
859
1551
859
1552
859
24
859
34
859
33
859
11
859
16
859
1541
859
1531
859
1522
859
1518
859
1509
859
1500
859
Batch Done, 4200
1501
859
1497
859
1500
859
1495
859
1488
859
1480
859
1485
859
1492
859
1502
859
1513
859
1520
859
1533
859
1474
859
1460
859
1467
859
1478
859
1476
859
155
859
47
859
1381
859
1389
859
1390
859
start_end changed
1384
859
1385
859
1374
859
1370
859
1367
859
1365
859
1357
859
1351
859
1341
859
1324
859
1321
859
1321
859
1303
859
1295
859
1269
859
124
859
151
859
70
859
1257
859
1250
859
1267
859
1184
859
1154
859
1134
859
1126
859
1114
859
1101
859
1090
859
1082
859
1032
859
1022
859
1066
859
995
859
979
859
966
859
966
859
989
859
8
859
949
859
925
859


1472
859
1489
859
1467
859
1461
859
1448
859
60
859
78
859
230
859
275
859
352
859
220
859
228
859
272
859
233
859
369
859
8
859
18
859
481
859
82
859
377
859
378
859
1444
859
start_end changed
328
859
306
859
209
859
209
859
881
859
170
859
1635
859
184
859
1575
859
1846
859
2317
859
392
859
358
859
354
859
344
859
344
859
344
859
302
859
288
859
282
859
269
859
250
859
242
859
218
859
228
859
224
859
188
859
176
859
160
859
130
859
124
859
136
859
103
859
86
859
77
859
94
859
Batch Done, 5200
83
859
93
859
150
859
136
859
110
859
236
859
245
859
47
859
50
859
63
859
62
859
170
859
154
859
128
859
29
859
35
859
45
859
63
859
98
859
98
859
23
859
241
859
42
859
60
859
65
859
59
859
38
859
36
859
640
859
541
859
524
859
531
859
520
859
425
859
417
859
49
859
38
859
17
859
82
859
132
859
140
859
148
859
22
859
19
859
12
859
12
859
133
859
135
859
28
859
42
859
46
859
49
859
49
859
55
859
874
859
864
859
856
859
847
859
834
859
809
859
69
859
79
859
81
859
717
859
707
859
712
859
727
859


139
859
188
859
202
859
198
859
166
859
171
859
22
859
280
859
243
859
221
859
233
859
2958
859
463
859
433
859
204
859
2814
859
2841
859
2848
859
27
859
37
859
82
859
99
859
65
859
61
859
89
859
92
859
37
859
23
859
15
859
25
859
35
859
35
859
2497
859
86
859
91
859
2436
859
2435
859
Batch Done, 6200
26
859
92
859
94
859
2246
859
2214
859
2213
859
2235
859
2254
859
2152
859
2165
859
241
859
257
859
270
859
237
859
382
859
397
859
426
859
441
859
start_end changed
476
859
590
859
580
859
581
859
616
859
654
859
646
859
680
859
693
859
721
859
725
859
768
859
817
859
828
859
232
859
187
859
187
859
189
859
165
859
164
859
3333
859
3338
859
52
859
77
859
73
859
90
859
84
859
69
859
1504
859
1503
859
75
859
47
859
52
859
58
859
66
859
9
859
100
859
10
859
22
859
111
859
64
859
14
859
29
859
20
859
28
859
193
859
209
859
256
859
256
859
458
859
9
859
495
859
16
859
1073
859
1040
859
104
859
1024
859
1010
859
129
859
991
859
967
859
118
859
272
859
921
859
835
859
32
859
27
859
19
859
71
85

IndexError: single positional indexer is out-of-bounds

In [17]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [ ]:
print(buildings)

In [18]:
road.to_file("output_road_1.geojson", driver="GeoJSON")